In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torchvision.models import resnet50

In [ ]:
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
THRESHOLD   = 0.5
MANIFEST    = "/home/jovyan/Features/manifest_test.csv"
TRAIN_CSV   = "/home/jovyan/Data/birdclef-2025/train.csv"
CKPT_PATH   = "resnet50_best_epoch.pt"

In [ ]:
# load test manifest
m_df = pd.read_csv(MANIFEST)

# load secondary_labels metadata
meta = pd.read_csv(TRAIN_CSV, usecols=["filename","secondary_labels"])
meta["recording_id"] = meta.filename.str.replace(r"\.ogg$", "", regex=True)
meta["sec_list"]     = meta.secondary_labels.fillna("").str.split()
sec_map = dict(zip(meta.recording_id, meta.sec_list))

# gather all labels from the test split
labels = set()
for _, row in m_df.iterrows():
    rid = row.chunk_id.split("_chk")[0]
    labels.add(row.primary_label)
    for sec in sec_map.get(rid, []):
        labels.add(sec)
classes = sorted(labels)

In [ ]:
def get_resnet50_multilabel(num_classes: int) -> nn.Module:
    model = resnet50(pretrained=False)
    # adapt first conv to accept 1‐channel (mel) input
    model.conv1 = nn.Conv2d(
        in_channels=1,
        out_channels=model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False
    )
    # replace final FC for multi‐label
    in_feat = model.fc.in_features
    model.fc = nn.Linear(in_feat, num_classes)
    return model

model = get_resnet50_multilabel(len(classes)).to(DEVICE)
ckpt = torch.load(CKPT_PATH, map_location=DEVICE)
model.load_state_dict(ckpt["model_state"])
model.eval()

In [ ]:
# sample one random chunk
sample = m_df.sample(1).iloc[0]
print("Inferring on chunk:", sample.chunk_id)

# load the augmented mel spectrogram
npz = np.load(sample.mel_aug_path)
mel = npz["mel"]  # shape: (n_mels, n_frames)

# convert to tensor [1,1,n_mels,n_frames]
x = torch.from_numpy(mel).unsqueeze(0).unsqueeze(0).float().to(DEVICE)

In [ ]:
with torch.no_grad():
    logits = model(x)               # [1, num_classes]
    probs  = torch.sigmoid(logits)[0]  # [num_classes]

In [ ]:
pred_idxs = (probs >= THRESHOLD).nonzero(as_tuple=False).flatten().tolist()
preds = [(classes[i], float(probs[i])) for i in pred_idxs]

print(f"\nPredictions (threshold ≥ {THRESHOLD}):")
for label, score in preds:
    print(f"  • {label}: {score:.3f}")